In [1]:
import numpy as np
import pandas as pd
import math
from functools import reduce

In [14]:
livetimes_ds3518 = pd.read_csv("livetimes_ds3518_2017Data.dat", delimiter="\t", names = ["Channel", "Livetime_ds3518"])
livetimes_ds3021 = pd.read_csv("livetimes_ds3021_2017Data.dat", delimiter="\t", names = ["Channel", "Livetime_ds3021"])

In [15]:
livetimes_merged = [livetimes_ds3518, livetimes_ds3021]

In [16]:
livetimes_all = reduce(lambda  left,right: pd.merge(left,right,on=['Channel'],
                                            how='outer'), livetimes_merged)
livetimes_all.head()

,Channel,Livetime_ds3518,Livetime_ds3021
0,1,1829414.0,2223603.0
1,2,1829414.0,2223603.0
2,3,1829414.0,2223603.0
3,4,1829414.0,2223603.0
4,5,1829414.0,2165463.0


In [17]:
# Add tower and floor information
# do get_tower before get_floor
def get_tower(row):
    channel = row["Channel"]
    tower = math.ceil(channel/52.0)
    return int(tower)
# requires get_tower first
def get_floor(row):
    channel = row["Channel"]
    tower = row["Tower"]
    channel_in_tower = channel - (52 * (tower - 1))
    floor = channel_in_tower % 13
    if (floor == 0):
        floor = 13
    return int(floor)

In [18]:
livetimes_all["Tower"] = livetimes_all.apply(get_tower, axis=1)
livetimes_all["Floor"] = livetimes_all.apply(get_floor, axis=1)

In [19]:
livetimes_all.head()

,Channel,Livetime_ds3518,Livetime_ds3021,Tower,Floor
0,1,1829414.0,2223603.0,1,1
1,2,1829414.0,2223603.0,1,2
2,3,1829414.0,2223603.0,1,3
3,4,1829414.0,2223603.0,1,4
4,5,1829414.0,2165463.0,1,5


In [20]:
# get livetimes for certain scenarios:

# lifetime of Odd/Even towers
def get_livetime_odd_tower(row):
    tower = int(row["Tower"])
    livetime = []
    if ((tower % 2) == 1):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        return sum(livetime)
    else:
        return 0
    
def get_livetime_even_tower(row):
    tower = int(row["Tower"])
    livetime = []
    if ((tower % 2) == 0):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        return sum(livetime)
    else:
        return 0
    
# Do the same for the floors
def get_livetime_odd_floor(row):
    floor = int(row["Floor"])
    livetime = []
    if ((floor % 2) == 1):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        return sum(livetime)
    else:
        return 0
    
def get_livetime_even_floor(row):
    floor = int(row["Floor"])
    livetime = []
    if ((floor % 2) == 0):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        return sum(livetime)
    else:
        return 0
#lifetime w/o Tower 12
def get_livetime_no_t12(row):
    tower = row["Tower"]
    livetime = []
    if (tower != 12):
        livetime.append(row["Livetime_ds3518"])
        livetime.append(row["Livetime_ds3021"])
        return sum(livetime)
    else:
        return 0

In [21]:
livetimes_all["Odd_Tower"] = livetimes_all.apply(get_livetime_odd_tower, axis=1)
livetimes_all["Even_Tower"] = livetimes_all.apply(get_livetime_even_tower, axis=1)
livetimes_all["Odd_Floor"] = livetimes_all.apply(get_livetime_odd_floor, axis=1)
livetimes_all["Even_Floor"] = livetimes_all.apply(get_livetime_even_floor, axis=1)
livetimes_all["No_T12"] = livetimes_all.apply(get_livetime_no_t12, axis=1)
livetimes_all.head()

,Channel,Livetime_ds3518,Livetime_ds3021,Tower,Floor,Odd_Tower,Even_Tower,Odd_Floor,Even_Floor,No_T12
0,1,1829414.0,2223603.0,1,1,4053017.0,0.0,4053017.0,0.0,4053017.0
1,2,1829414.0,2223603.0,1,2,4053017.0,0.0,0.0,4053017.0,4053017.0
2,3,1829414.0,2223603.0,1,3,4053017.0,0.0,4053017.0,0.0,4053017.0
3,4,1829414.0,2223603.0,1,4,4053017.0,0.0,0.0,4053017.0,4053017.0
4,5,1829414.0,2165463.0,1,5,3994877.0,0.0,3994877.0,0.0,3994877.0


In [22]:
# Turn livetime into exposure
def convert_to_exposure(livetime):
    return livetime * (0.75 / (60 * 60 * 24 * 365.25))

In [25]:
#lifetimes to use in analysis
print ("3018 exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[1])))
print ("3021 exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[2])))
print ("Total exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[2] + livetimes_all.sum(axis=0)[1])))
print ("Odd Tower exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[5])))
print ("Even Tower exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[6])))
print ("Odd Floor exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[7])))
print ("Even Floor exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[8])))
print ("No T12 exposure: %s" %(convert_to_exposure(livetimes_all.sum(axis=0)[9])))

3018 exposure: 37.70302180774203
3021 exposure: 49.34772523100616
Total exposure: 87.05074703874818
Odd Tower exposure: 45.95211843581261
Even Tower exposure: 41.098628602935584
Odd Floor exposure: 45.97019961118716
Even Floor exposure: 41.08054742756103
No T12 exposure: 82.61646703171344
